In [1]:
"""import findspark
findspark.init()
import pyspark as ps
from pyspark.sql import SparkSession
import string
import nltk
from nltk.corpus import stopwords
import pandas as pd"""

'import findspark\nfindspark.init()\nimport pyspark as ps\nfrom pyspark.sql import SparkSession\nimport string\nimport nltk\nfrom nltk.corpus import stopwords\nimport pandas as pd'

In [2]:
import random
import findspark
findspark.init()
import pyspark as ps
from pyspark.sql import SparkSession

In [3]:
import string
import nltk
from nltk.corpus import stopwords
import pandas as pd

In [4]:
spark1 = SparkSession.builder.appName('Ops').getOrCreate()

In [45]:
def remove_punc(str):
    table = str.maketrans('', '', string.punctuation)
    words=str.split()
    stripped = [w.translate(table) for w in words]
    return stripped
def intersection(lst1,lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3
def disjoin(lst1,lst2):
    lst3 = [value for value in lst1 if value not in lst2]
    lst4 = [value for value in lst2 if value not in lst1]
    return lst3 + lst4
def filter_left(lst1,lst2):
    lst3 = [value for value in lst1 if value not in lst2]
    return lst3

In [47]:
list1 = ['a','b','c','d']
list2 = ['b','c','d','e']
print(intersection(list1,list2))
print(disjoin(list1,list2))
print(filter_left(list1,list2))

['b', 'c', 'd']
['a', 'e']
['a']


In [7]:
path = "user.json"
user_df = spark1.read.json(path)
user_df.printSchema()
user_df.createOrReplaceTempView("user")

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)



In [8]:
path = "business.json"
business_df = spark1.read.json(path)
business_df.printSchema()
business_df.createOrReplaceTempView("business")

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

In [9]:
path = "review.json"
review_df = spark1.read.json(path)
review_df.printSchema()
review_df.createOrReplaceTempView("review")

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



In [10]:
phoenix_1_stars = spark1.sql("SELECT business.business_id, categories, text FROM business JOIN review ON business.business_id=review.business_id WHERE city = 'Phoenix' and review.stars = 1").toPandas()
phoenix_2_stars = spark1.sql("SELECT business.business_id, categories, text FROM business JOIN review ON business.business_id=review.business_id WHERE city = 'Phoenix' and business.stars = 2").toPandas()
phoenix_3_stars = spark1.sql("SELECT business.business_id, categories, text FROM business JOIN review ON business.business_id=review.business_id WHERE city = 'Phoenix' and business.stars = 3").toPandas()
phoenix_4_stars = spark1.sql("SELECT business.business_id, categories, text FROM business JOIN review ON business.business_id=review.business_id WHERE city = 'Phoenix' and business.stars = 4").toPandas()
phoenix_5_stars = spark1.sql("SELECT business.business_id, categories, text FROM business JOIN review ON business.business_id=review.business_id WHERE city = 'Phoenix' and business.stars = 5").toPandas()

In [11]:
print(phoenix_1_stars.shape[0])
print(phoenix_2_stars.shape[0])
print(phoenix_3_stars.shape[0])
print(phoenix_4_stars.shape[0])
print(phoenix_5_stars.shape[0])

118901
22939
73511
227337
51881


In [12]:
phoenix_1_stars['text'] = phoenix_1_stars['text'].apply(lambda x: x.lower())
phoenix_2_stars['text'] = phoenix_2_stars['text'].apply(lambda x: x.lower())
phoenix_3_stars['text'] = phoenix_3_stars['text'].apply(lambda x: x.lower())
phoenix_4_stars['text'] = phoenix_4_stars['text'].apply(lambda x: x.lower())
phoenix_5_stars['text'] = phoenix_5_stars['text'].apply(lambda x: x.lower())

In [13]:
phoenix_1_stars['text'] = phoenix_1_stars['text'].apply(lambda x: remove_punc(x))
phoenix_2_stars['text'] = phoenix_2_stars['text'].apply(lambda x: remove_punc(x))
phoenix_3_stars['text'] = phoenix_3_stars['text'].apply(lambda x: remove_punc(x))
phoenix_4_stars['text'] = phoenix_4_stars['text'].apply(lambda x: remove_punc(x))
phoenix_5_stars['text'] = phoenix_5_stars['text'].apply(lambda x: remove_punc(x))

In [14]:
#Removing stop words
nltk.download('stopwords')
stop = stopwords.words('english')

def rem_stop_words(txt):
    return ' '.join([word for word in txt if word not in stop])

[nltk_data] Downloading package stopwords to /home/carter/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
phoenix_1_stars['text'] = phoenix_1_stars['text'].apply(lambda x: rem_stop_words(x))
phoenix_2_stars['text'] = phoenix_2_stars['text'].apply(lambda x: rem_stop_words(x))
phoenix_3_stars['text'] = phoenix_3_stars['text'].apply(lambda x: rem_stop_words(x))
phoenix_4_stars['text'] = phoenix_4_stars['text'].apply(lambda x: rem_stop_words(x))
phoenix_5_stars['text'] = phoenix_5_stars['text'].apply(lambda x: rem_stop_words(x))

In [16]:
#Word Frequency
freq1 = pd.Series(' '.join(phoenix_1_stars['text']).split()).value_counts().sort_values(ascending=False)

In [17]:
freq2 = pd.Series(' '.join(phoenix_2_stars['text']).split()).value_counts().sort_values(ascending=False)
freq3 = pd.Series(' '.join(phoenix_3_stars['text']).split()).value_counts().sort_values(ascending=False)
freq4 = pd.Series(' '.join(phoenix_4_stars['text']).split()).value_counts().sort_values(ascending=False)
freq5 = pd.Series(' '.join(phoenix_5_stars['text']).split()).value_counts().sort_values(ascending=False)


In [55]:
print("1 star reviews")
p1_star_word_list = []
for x in freq1[:1000].iteritems():
    print(x)
    p1_star_word_list.append(x[0])
#print(p1_star_word_list)

1 star reviews
('would', 69778)
('get', 63235)
('back', 58273)
('time', 56375)
('service', 55810)
('one', 55526)
('place', 49210)
('said', 46835)
('told', 46670)
('like', 46281)
('never', 45995)
('food', 44412)
('even', 41514)
('dont', 41072)
('go', 40768)
('us', 39190)
('didnt', 34737)
('could', 33042)
('got', 33011)
('asked', 31190)
('car', 30846)
('called', 29499)
('went', 29019)
('good', 28240)
('order', 28037)
('customer', 27865)
('minutes', 26720)
('people', 26526)
('call', 26248)
('im', 25528)
('going', 24925)
('know', 24864)
('came', 24804)
('another', 23994)
('first', 23769)
('day', 22642)
('ever', 22472)
('take', 21877)
('manager', 21826)
('company', 21758)
('still', 21321)
('give', 21000)
('come', 20817)
('took', 20577)
('2', 20480)
('make', 20326)
('really', 19929)
('way', 19898)
('work', 19864)
('two', 19591)
('bad', 19307)
('also', 19304)
('new', 19020)
('experience', 18969)
('money', 18874)
('ordered', 18836)
('well', 18642)
('business', 18537)
('want', 18431)
('left', 1

In [56]:
print("2 star reviews")
p2_star_word_list = []
for x in freq2[:1000].iteritems():
    print(x)
    p2_star_word_list.append(x[0])

2 star reviews
('get', 12023)
('would', 11675)
('time', 10152)
('one', 9836)
('service', 9769)
('back', 8889)
('car', 8795)
('like', 7901)
('place', 7839)
('dont', 7504)
('told', 7193)
('go', 6976)
('never', 6917)
('said', 6820)
('even', 6767)
('got', 5776)
('didnt', 5425)
('us', 5382)
('could', 5375)
('good', 5374)
('people', 5261)
('called', 5044)
('food', 5003)
('im', 4792)
('call', 4778)
('went', 4710)
('customer', 4591)
('order', 4378)
('day', 4375)
('work', 4314)
('asked', 4167)
('know', 4135)
('going', 4116)
('new', 4052)
('really', 3878)
('come', 3873)
('another', 3863)
('great', 3853)
('still', 3844)
('company', 3834)
('also', 3834)
('location', 3830)
('minutes', 3814)
('manager', 3788)
('first', 3774)
('2', 3743)
('give', 3671)
('ever', 3671)
('ive', 3625)
('staff', 3621)
('take', 3617)
('came', 3564)
('make', 3512)
('well', 3470)
('two', 3401)
('always', 3389)
('took', 3339)
('way', 3318)
('experience', 3288)
('want', 3187)
('office', 3108)
('bad', 3068)
('times', 3000)
('se

In [57]:
print("3 star reviews")
p3_star_word_list =[]
for x in freq3[:1000].iteritems():
    print(x)
    p3_star_word_list.append(x[0])

3 star reviews
('food', 34971)
('good', 32252)
('place', 31686)
('get', 29491)
('time', 28359)
('service', 27908)
('like', 27384)
('would', 26389)
('one', 26380)
('great', 26030)
('back', 23996)
('go', 21299)
('dont', 17251)
('really', 16952)
('even', 16667)
('us', 15986)
('got', 15488)
('never', 14671)
('didnt', 14034)
('im', 13593)
('staff', 13445)
('also', 13299)
('nice', 13129)
('order', 13067)
('went', 13044)
('always', 12969)
('said', 12893)
('could', 12850)
('well', 12516)
('people', 12056)
('first', 11543)
('told', 11489)
('came', 11441)
('going', 11288)
('ive', 11046)
('know', 10659)
('location', 10525)
('ordered', 10293)
('come', 10264)
('much', 10106)
('experience', 10077)
('make', 9881)
('friendly', 9854)
('way', 9818)
('two', 9602)
('minutes', 9569)
('day', 9549)
('new', 9436)
('love', 9327)
('asked', 9321)
('2', 9310)
('little', 9254)
('customer', 9241)
('better', 9114)
('still', 8773)
('best', 8767)
('bad', 8702)
('want', 8679)
('take', 8656)
('see', 8654)
('work', 8645)

In [58]:
print("4 star reviews")
p4_star_word_list = []
for x in freq4[:1000].iteritems():
    print(x)
    p4_star_word_list.append(x[0])

4 star reviews
('food', 129708)
('place', 125469)
('good', 121451)
('great', 117406)
('service', 78857)
('like', 78074)
('time', 74131)
('back', 69499)
('one', 68170)
('get', 67062)
('would', 62232)
('go', 58807)
('really', 58377)
('love', 47186)
('also', 46664)
('best', 42704)
('us', 41147)
('well', 40891)
('got', 40747)
('nice', 40626)
('dont', 39868)
('always', 39621)
('im', 38981)
('ordered', 38856)
('even', 37900)
('ive', 36755)
('little', 36590)
('try', 36522)
('delicious', 36410)
('friendly', 35633)
('restaurant', 35326)
('chicken', 34757)
('first', 34709)
('staff', 34680)
('order', 34640)
('menu', 34497)
('amazing', 33891)
('came', 32623)
('didnt', 31388)
('definitely', 30811)
('went', 30380)
('could', 29257)
('never', 28960)
('come', 28918)
('much', 28802)
('pizza', 28374)
('make', 27712)
('people', 27689)
('made', 27513)
('bar', 27204)
('wait', 27098)
('experience', 26138)
('phoenix', 25792)
('know', 25589)
('fresh', 25517)
('going', 25269)
('way', 24808)
('pretty', 24758)
('

In [59]:
print("5 star reviews")
p5_star_word_list = []
for x in freq5[:1000].iteritems():
    print(x)
    p5_star_word_list.append(x[0])

5 star reviews
('great', 24501)
('time', 19188)
('would', 15312)
('recommend', 13788)
('work', 13641)
('service', 13062)
('get', 12908)
('place', 11896)
('back', 11100)
('best', 11067)
('one', 10702)
('good', 10631)
('like', 10124)
('amazing', 9579)
('us', 9049)
('job', 9020)
('also', 8989)
('highly', 8895)
('new', 8830)
('even', 8744)
('really', 8677)
('go', 8667)
('friendly', 8204)
('well', 8004)
('first', 7859)
('professional', 7806)
('day', 7716)
('always', 7536)
('home', 7483)
('experience', 7381)
('got', 7267)
('definitely', 7187)
('ive', 7159)
('made', 7135)
('could', 6873)
('love', 6814)
('im', 6739)
('done', 6735)
('came', 6631)
('company', 6518)
('staff', 6405)
('make', 6256)
('years', 6088)
('going', 6031)
('know', 5981)
('dr', 5954)
('much', 5885)
('right', 5816)
('everything', 5695)
('took', 5664)
('never', 5610)
('went', 5548)
('thank', 5514)
('found', 5475)
('house', 5461)
('called', 5452)
('car', 5321)
('nice', 5294)
('dont', 5184)
('care', 5157)
('way', 5148)
('sure', 

In [61]:
common_words = intersection(p1_star_word_list,p2_star_word_list)
#print("1&2 length: ",len(common_words), " ", common_words)
common_words = intersection(common_words,p3_star_word_list)
#print("&3",common_words)
common_words = intersection(common_words,p4_star_word_list)
#print("&4",common_words)
common_words = intersection(common_words,p5_star_word_list)
print(len(common_words))
print("common words: ",common_words)
unique_1 = filter_left(p1_star_word_list,p2_star_word_list)
unique_1 = filter_left(unique_1,p3_star_word_list)
unique_1 = filter_left(unique_1,p4_star_word_list)
unique_1 = filter_left(unique_1,p5_star_word_list)

unique_2 = filter_left(p2_star_word_list,p1_star_word_list)
unique_2 = filter_left(unique_2,p3_star_word_list)
unique_2 = filter_left(unique_2,p4_star_word_list)
unique_2 = filter_left(unique_2,p5_star_word_list)

unique_3 = filter_left(p3_star_word_list,p1_star_word_list)
unique_3 = filter_left(unique_3,p2_star_word_list)
unique_3 = filter_left(unique_3,p4_star_word_list)
unique_3 = filter_left(unique_3,p5_star_word_list)

unique_4 = filter_left(p4_star_word_list,p1_star_word_list)
unique_4 = filter_left(unique_4,p2_star_word_list)
unique_4 = filter_left(unique_4,p3_star_word_list)
unique_4 = filter_left(unique_4,p5_star_word_list)

unique_5 = filter_left(p5_star_word_list,p1_star_word_list)
unique_5 = filter_left(unique_5,p2_star_word_list)
unique_5 = filter_left(unique_5,p3_star_word_list)
unique_5 = filter_left(unique_5,p4_star_word_list)

print("Words unique to 1 star reviews: ", unique_1)
print()
print("Words unique to 2 star reviews: ", unique_2)
print()
print("Words unique to 3 star reviews: ", unique_3)
print()
print("Words unique to 4 star reviews: ", unique_4)
print()
print("Words unique to 5 star reviews: ", unique_5)

565
common words:  ['would', 'get', 'back', 'time', 'service', 'one', 'place', 'said', 'told', 'like', 'never', 'food', 'even', 'dont', 'go', 'us', 'didnt', 'could', 'got', 'asked', 'car', 'called', 'went', 'good', 'order', 'customer', 'minutes', 'people', 'call', 'im', 'going', 'know', 'came', 'another', 'first', 'day', 'ever', 'take', 'company', 'still', 'give', 'come', 'took', '2', 'make', 'really', 'way', 'work', 'two', 'bad', 'also', 'new', 'experience', 'money', 'ordered', 'well', 'business', 'want', 'left', 'say', 'see', 'ive', 'phone', 'made', 'times', '3', 'wanted', 'right', 'wait', 'since', 'last', 'pay', 'much', 'staff', 'home', 'location', 'around', 'next', 'better', 'later', 'away', 'nothing', 'done', 'wasnt', 'someone', 'tell', 'put', 'great', 'need', 'care', 'store', 'hour', 'finally', 'anything', 'sure', 'find', 'cant', 'long', 'waiting', 'hours', 'restaurant', 'days', 'use', 'something', 'think', 'check', 'front', '5', 'ask', 'office', 'years', 'try', 'every', 'price',

In [72]:
print("Manually assigning words to industries/service trends for 1 star reviews")
classified_1 = []
for i in range(0,len(unique_1)):
    print("What catagory does ", unique_1[i], " fall into")
    u_in = input()
    classified_1.append(u_in)
for i in range(0,len(classified_1)):
    print("word: ", unique_1[i], "assigned to: ", classified_1[i])

Manually assigning words to industries/service trends for 1 star reviews
What catagory does  sick  fall into
illness
What catagory does  shame  fall into
dissatisfaction
What catagory does  needless  fall into
waste
What catagory does  ignored  fall into
bad service
What catagory does  nasty  fall into
negative
What catagory does  claim  fall into
false advertising
What catagory does  continued  fall into
misc
What catagory does  scam  fall into
waste
What catagory does  cox  fall into
misc
What catagory does  apology  fall into
apology
What catagory does  tires  fall into
attempt
What catagory does  child  fall into
children
What catagory does  dollar  fall into
price -cheap
What catagory does  figured  fall into
expectations
What catagory does  shouldnt  fall into
negative
What catagory does  arrive  fall into
time
What catagory does  removed  fall into
negative
What catagory does  poorly  fall into
negative
What catagory does  cancelled  fall into
negative
What catagory does  allowe

In [73]:
print("Manually assigning words to industries/service trends for 2 star reviews")
classified_2 = []
for i in range(0,len(unique_2)):
    print("What catagory does ", unique_2[i], " fall into")
    u_in = input()
    classified_2.append(u_in)
for i in range(0,len(classified_2)):
    print("word: ", unique_2[i], "assigned to: ", classified_2[i])

Manually assigning words to industries/service trends for 2 star reviews
What catagory does  hertz  fall into
automotive
What catagory does  nissan  fall into
automotive
What catagory does  cab  fall into
transportation
What catagory does  walmart  fall into
retail
What catagory does  apartments  fall into
housing
What catagory does  camelback  fall into
merchandise
What catagory does  mcdonalds  fall into
chain restaurant
What catagory does  salesman  fall into
salesman
What catagory does  sons  fall into
children
What catagory does  ford  fall into
automotive
What catagory does  finance  fall into
finance
What catagory does  neighbors  fall into
location
What catagory does  parker  fall into
misc
What catagory does  express  fall into
time
What catagory does  station  fall into
misc -transportaion?
What catagory does  keys  fall into
auto/housing?
What catagory does  loan  fall into
finance
What catagory does  managers  fall into
staff
What catagory does  lines  fall into
wait
What c

In [74]:
print("Manually assigning words to industries/service trends for 2 star reviews")
classified_3 = []
for i in range(0,len(unique_3)):
    print("What catagory does ", unique_3[i], " fall into")
    u_in = input()
    classified_3.append(u_in)
for i in range(0,len(classified_3)):
    print("word: ", unique_3[i], "assigned to: ", classified_3[i])

Manually assigning words to industries/service trends for 2 star reviews
What catagory does  movie  fall into
movie
What catagory does  theater  fall into
movie
What catagory does  buffet  fall into
restaurant
What catagory does  resort  fall into
hotel
What catagory does  chipotle  fall into
chain restaurant
What catagory does  stores  fall into
retail
What catagory does  chain  fall into
chain
What catagory does  sports  fall into
sports
What catagory does  dresses  fall into
clothing
What catagory does  tickets  fall into
misc
What catagory does  lobby  fall into
misc
What catagory does  venue  fall into
venue
What catagory does  kid  fall into
children
What catagory does  chairs  fall into
furniture
What catagory does  section  fall into
seating
What catagory does  tvs  fall into
entertainment
What catagory does  frozen  fall into
temperature
What catagory does  nights  fall into
night
What catagory does  watching  fall into
entertainment
What catagory does  otherwise  fall into
mi

In [75]:
print("Manually assigning words to industries/service trends for 2 star reviews")
classified_4 = []
for i in range(0,len(unique_4)):
    print("What catagory does ", unique_4[i], " fall into")
    u_in = input()
    classified_4.append(u_in)
for i in range(0,len(classified_4)):
    print("word: ", unique_4[i], "assigned to: ", classified_4[i])

Manually assigning words to industries/service trends for 2 star reviews
What catagory does  chocolate  fall into
dessert
What catagory does  brunch  fall into
restaurant
What catagory does  ambiance  fall into
atmosphere
What catagory does  flavorful  fall into
good tasting food
What catagory does  crust  fall into
food
What catagory does  italian  fall into
restaurant - italian
What catagory does  pho  fall into
restaurant - asian
What catagory does  crispy  fall into
food
What catagory does  vibe  fall into
atmosphere
What catagory does  pasta  fall into
restaurant - italian
What catagory does  authentic  fall into
restaurant
What catagory does  salmon  fall into
food
What catagory does  crab  fall into
food
What catagory does  butter  fall into
food
What catagory does  vegan  fall into
food
What catagory does  garlic  fall into
food
What catagory does  toast  fall into
food
What catagory does  salads  fall into
food
What catagory does  thai  fall into
restaurant - asian
What catago

In [76]:
print("Manually assigning words to industries/service trends for 2 star reviews")
classified_5 = []
for i in range(0,len(unique_5)):
    print("What catagory does ", unique_5[i], " fall into")
    u_in = input()
    classified_5.append(u_in)
for i in range(0,len(classified_5)):
    print("word: ", unique_5[i], "assigned to: ", classified_5[i])

Manually assigning words to industries/service trends for 2 star reviews
What catagory does  brisket  fall into
restaurant - bbq
What catagory does  thorough  fall into
misc
What catagory does  body  fall into
misc
What catagory does  classes  fall into
education
What catagory does  results  fall into
results
What catagory does  garage  fall into
parking
What catagory does  studio  fall into
art
What catagory does  pricing  fall into
pricing
What catagory does  courteous  fall into
customer service
What catagory does  shes  fall into
staff
What catagory does  john  fall into
name
What catagory does  products  fall into
products
What catagory does  mike  fall into
name
What catagory does  auto  fall into
misc
What catagory does  dental  fall into
dental
What catagory does  fit  fall into
misc
What catagory does  clients  fall into
customers
What catagory does  knowledge  fall into
staff
What catagory does  yoga  fall into
fitness
What catagory does  affordable  fall into
pricing
What ca

What catagory does  listened  fall into
customer service
What catagory does  flowers  fall into
flowers
What catagory does  justin  fall into
name
What catagory does  pest  fall into
negative
What catagory does  feels  fall into
misc
What catagory does  beginning  fall into
misc
What catagory does  lower  fall into
pricing
What catagory does  tint  fall into
automotive
word:  brisket assigned to:  restaurant - bbq
word:  thorough assigned to:  misc
word:  body assigned to:  misc
word:  classes assigned to:  education
word:  results assigned to:  results
word:  garage assigned to:  parking
word:  studio assigned to:  art
word:  pricing assigned to:  pricing
word:  courteous assigned to:  customer service
word:  shes assigned to:  staff
word:  john assigned to:  name
word:  products assigned to:  products
word:  mike assigned to:  name
word:  auto assigned to:  misc
word:  dental assigned to:  dental
word:  fit assigned to:  misc
word:  clients assigned to:  customers
word:  knowledge as

In [77]:
#get the count of classifications
counts_1 = []
counts_2 = []
counts_3 = []
counts_4 = []
counts_5 = []
print("aggregating data for 1 star reviews:")
classified_1_set = list(set(classified_1))
for i in range(0,len(classified_1_set)):
    counts_1.append(classified_1.count(classified_1_set[i]))
    print(classified_1_set[i], ": ", counts_1[i])

print("\naggregating data for 2 star reviews:")
classified_2_set = list(set(classified_2))
for i in range(0,len(classified_2_set)):
    counts_2.append(classified_2.count(classified_2_set[i]))
    print(classified_2_set[i], ": ", counts_2[i])

print("\naggregating data for 3 star reviews:")
classified_3_set = list(set(classified_3))
for i in range(0,len(classified_3_set)):
    counts_3.append(classified_3.count(classified_3_set[i]))
    print(classified_3_set[i], ": ", counts_3[i])

print("\naggregating data for 4 star reviews:")
classified_4_set = list(set(classified_4))
for i in range(0,len(classified_4_set)):
    counts_4.append(classified_4.count(classified_4_set[i]))
    print(classified_4_set[i], ": ", counts_4[i])

print("\naggregating data for 5 star reviews:")
classified_5_set = list(set(classified_5))
for i in range(0,len(classified_5_set)):
    counts_5.append(classified_5.count(classified_5_set[i]))
    print(classified_5_set[i], ": ", counts_5[i])

aggregating data for 1 star reviews:
expectations :  2
apology :  2
negative :  5
optometry :  1
time :  1
wait :  1
dissatisfaction :  1
bad service :  1
misc :  2
waste :  3
illness :  1
children :  1
false advertising :  1
permission :  1
attempt :  1
price -cheap :  1
aggregating data for 2 star reviews:
staff :  1
misc - automotive? :  1
merchandise :  1
automotive :  6
wait :  1
chain restaurant :  1
auto/housing? :  1
retail :  1
time :  1
salesman :  1
paying extra :  1
misc :  12
misc -transportaion? :  1
communication :  1
complaints :  1
pharmacy :  2
poor grammar :  1
children :  1
transportation :  1
housing :  10
location :  1
finance :  2
aggregating data for 3 star reviews:
chain restaurant :  1
sports :  1
seating :  1
retail :  1
temperature :  1
misc :  5
movie :  2
furniture :  1
clothing :  1
children :  1
entertainment :  2
chain :  1
hotel :  1
venue :  1
restaurant :  1
night :  1
aggregating data for 4 star reviews:
portions :  1
food or atmosphere :  1
restaur

In [89]:
#bubble sort the lists, then report the top 10 from each list
def bubblesort(arr1,arr2):
    n = len(arr1)
 
    # Traverse through all array elements
    for i in range(n):
 
        # Last i elements are already in place
        for j in range(0, n-i-1):
 
            # traverse the array from 0 to n-i-1
            # Swap if the element found is greater
            # than the next element
            if arr2[j] < arr2[j+1] :
                arr1[j], arr1[j+1] = arr1[j+1], arr1[j]
                arr2[j], arr2[j+1] = arr2[j+1], arr2[j]

In [90]:
bubblesort(classified_1_set,counts_1)
bs_2 = bubblesort(classified_2_set,counts_2)
bs_3 = bubblesort(classified_3_set,counts_3)
bs_4 = bubblesort(classified_4_set,counts_4)
bs_5 = bubblesort(classified_5_set,counts_5)
print("top 10 classifications for 1 star reviews: ")
for i in range(0,11):
    print(classified_1_set[i],": ", counts_1[i])
print("\ntop 10 classifications for 2 star reviews: ")
for i in range(0,11):
    print(classified_2_set[i],": ", counts_2[i])
print("\ntop 10 classifications for 3 star reviews: ")
for i in range(0,11):
    print(classified_3_set[i],": ", counts_3[i])
print("\ntop 10 classifications for 4 star reviews: ")
for i in range(0,11):
    print(classified_4_set[i],": ", counts_4[i])
print("\ntop 10 classifications for 5 star reviews: ")
for i in range(0,11):
    print(classified_5_set[i],": ", counts_5[i])

top 10 classifications for 1 star reviews: 
negative :  5
waste :  3
apology :  2
expectations :  2
misc :  2
attempt :  1
bad service :  1
children :  1
dissatisfaction :  1
false advertising :  1
illness :  1

top 10 classifications for 2 star reviews: 
misc :  12
housing :  10
automotive :  6
finance :  2
pharmacy :  2
auto/housing? :  1
chain restaurant :  1
children :  1
communication :  1
complaints :  1
location :  1

top 10 classifications for 3 star reviews: 
misc :  5
entertainment :  2
movie :  2
chain :  1
chain restaurant :  1
children :  1
clothing :  1
furniture :  1
hotel :  1
night :  1
restaurant :  1

top 10 classifications for 4 star reviews: 
food :  74
misc :  28
atmosphere :  11
drinks :  5
restaurant :  4
geography :  3
restaurant - italian :  3
dessert :  2
good tasting food :  2
restaurant - asian :  2
children :  1

top 10 classifications for 5 star reviews: 
misc :  49
customer service :  20
name :  20
positive :  10
service :  10
staff :  10
housing :  6
ne

In [92]:
print("Conclusion: ")
print("We were able to generate a list fot he 1000 most frequent words \
      per star rating. That is we genereated the 1000 most frequent words \
      from 1 star revies, then a separate list for 2 star reviews etc. \
      We then looked for similarities among those lists by performing an \
      intersect operation between the lists. We decided to then look for \
      words from the frequency lists that were unique to that star rating. \
      Next we manually classified the unique lists and tallied the classification \
      entries and reported the top 10 from each star ranking. We decided to manually \
      classify each word rather than using an algorithm to do so because classification \
      of text is difficult, and a human will have a far greater understanding of context \
      than an automated process. If we were to persue this further, an algorithm for classification \
      would be in order, but as it is, selecting unique words from the list of frequent words reduced \
      the amount of data from millions of words to a couple hundred words, something managable for a \
      person to fill out. Surveying additional people to classify the word lists into catagories would \
      help reduce individual bias, and thus be a good next step as well as a classification algorithm.")
print("The idea being that if a word has a high frequency in all catagories, that it is probably not significant. \
       We feel that the method is statistically significant because if a word has a high frequency in one catagory, \
       at least frequent enough to make it into the list of top 1000 most frequently used words for a rating catagory, \
       but not make it into any of the other frequency lists, then it is uniquely predominant to that catagory.")
print("We chose to examine the top 1000 frequnt words per star rating arbitrarily. 100 was too few to \
       obtain meaningful results, and 1000 seems to capture enough data to suit our purposes, though \
       only investigating on this one sample size could skew the data somewhat due to our unique constraing. \
       A solution to this would be to perform the same steps on different samples sizes and find an optimal \
       number for frequent words to examine. As it stands, I believe that 1000 most frequent words can and does \
       provide insight into our question of what do yelp reviewers in pheonix arizona value most.")
print("Results for 1 star reviews: ")
print("The main aspects of 1 star reviews in Peonix were generally negative words like 'shame', 'nasty', 'poorly' \
       and wastefulness, indicating a waste of money or time. ")
print("Results for 2 star reviews: ")
print("Housing and automotive services seem to be uniquely predominant in the 2 star reviews for pheonix. Pharmacies \
       and finance also showed up multiple times.")
print("Results for 3 star reviews: ")
print("The prodominant word classifications for 3 star reviews were entertainment such as sports, and several instances \
      words directly relating to movies/cinema")
print("Results for 4 star reviews: ")
print("Four star reviews were mostly food related. Italian and Asian foods were especially common in the word rankings. \
       Comments about ingredients and atmosphere appear in this ranking much higher than any other ratings bin. ")
print("Results for 1 star reviews: ")
print("Five star reviews heavily emphasised customer service, good service, staff, and individual names. These reviews \
       had many positive words such as 'happy', 'love', etc. Pricing was also mentioned more here than in other \
       catagories, and many of the words were service related, such as 'replace' and 'project'. As for industries, \
       service industries, housing, art, education were mentioned far more than other industries such as restaurants.")

Conclusion: 
We were able to generate a list fot he 1000 most frequent words       per star rating. That is we genereated the 1000 most frequent words       from 1 star revies, then a separate list for 2 star reviews etc.       We then looked for similarities among those lists by performing an       intersect operation between the lists. We decided to then look for       words from the frequency lists that were unique to that star rating.       Next we manually classified the unique lists and tallied the classification       entries and reported the top 10 from each star ranking. We decided to manually       classify each word rather than using an algorithm to do so because classification       of text is difficult, and a human will have a far greater understanding of context       than an automated process. If we were to persue this further, an algorithm for classification       would be in order, but as it is, selecting unique words from the list of frequent words reduced       the a